<a href="https://colab.research.google.com/github/arpit0891/Python-Algorithms/blob/master/Data_Simulation_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install simpy

In [0]:
import random

In [0]:
import simpy

In [0]:
class Car(object): 
  def __init__(self, env): 
    self.env = env
    self.action = env.process(self.run()) 
  def run(self):
    while True: 
     print('Start parking and charging at %d' % self.env.now) 
     charge_duration = 5 
     yield self.env.process(self.charge(charge_duration)) 
     print('Start driving at %d' % self.env.now)
     trip_duration = 2 
     yield self.env.timeout(trip_duration)
  def charge(self, duration):
     yield self.env.timeout(duration)
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [0]:
def car(env, name, bcs, driving_time, charge_duration):
  yield env.timeout(driving_time)
  print('%s arriving at %d' % (name, env.now))
  with bcs.request() as req:
    yield req
    print('%s starting to charge at %s' % (name, env.now))
    yield env.timeout(charge_duration)
    print('%s leaving the bcs at %s' % (name, env.now))
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range (4):
  env.process(car(env, 'Car %d' % i, bcs, i*2, 5)) 
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [0]:
def example(env):
  event = simpy.events.Timeout(env, delay=1, value=42)
  value = yield event
  print('now=%d, value=%d' % (env.now, value))
env= simpy.Environment()
example_gen = example(env)
p = simpy.events.Process(env, example_gen)
env.run()

now=1, value=42


In [0]:
def subfunc(env):
  print(evc.active_process)
def my_proc(env):
  while True:
    print(env.active_process)
    subfunc(env)
    yield env.timeout(3)
env = simpy.Environment()
p1 = env.process(my_proc(env))
env.active_process
env.step() 
env.active_process

<Process(my_proc) object at 0x7f2a4b9ac0b8>


In [0]:
from random import seed, randint
seed(23)
class EV:
  def __init__(self, env):
    self.env = env 
    self.drive_proc = env.process(self.drive(env)) 
    self.bat_ctrl_proc = env.process(self.bat_ctrl(env)) 
    self.bat_ctrl_reactivate = env.event()
  def drive(self, env):
     while True:
      yield env.timeout(randint(20, 40)) 
      print('Start parking at', env.now) 
      self.bat_ctrl_reactivate.succeed()
      self.bat_ctrl_reactivate = env.event()
      yield env.timeout(randint(60, 360))
      print('Stop parking at', env.now)
  def bat_ctrl(self, env):
    while True:
      print('Bat. ctrl. passivating at', env.now)
      yield self.bat_ctrl_reactivate
      print('Bat. ctrl. reactivated at', env.now)
      yield env.timeout(randint(30, 90))
env = simpy.Environment()
ev = EV(env)
env.run(until=150)

Bat. ctrl. passivating at 0
Start parking at 29
Bat. ctrl. reactivated at 29
Bat. ctrl. passivating at 60
Stop parking at 131


In [0]:
def signal(env):
  while True:
    print("GREEN LIGHT " ,env.now)
    duration1=30
    yield env.timeout(duration1)

    print("YELLOW LIGHT ", env.now)
    duration2=5
    yield env.timeout(duration2)

    print("RED LIGHT " ,env.now)
    duration3=85
    yield env.timeout(duration3)
env=simpy.Environment()
env.process(signal(env))
env.run(until=500)

GREEN LIGHT  0
YELLOW LIGHT  30
RED LIGHT  35
GREEN LIGHT  120
YELLOW LIGHT  150
RED LIGHT  155
GREEN LIGHT  240
YELLOW LIGHT  270
RED LIGHT  275
GREEN LIGHT  360
YELLOW LIGHT  390
RED LIGHT  395
GREEN LIGHT  480


In [0]:
from collections import deque
from numpy import random
import simpy
from simpy.util import start_delayed


class Struct(object):
   def __init__(self, **kwargs):
      self.__dict__.update(kwargs)
random.seed([1, 2, 3])
#specify the initial data
end_time= 200.0
arrival_rate= 0.05
t_interarrival_mean= 2.0 / arrival_rate
t_green= 30.0
t_red= 90.0
t_depart_left= 1.6; t_depart_mode= 2.0; t_depart_right= 2.4
queue= deque()
arrival_count= departure_count= 0
Q_stats= Struct(count=0, cars_waiting=0)
W_stats= Struct(count=0, waiting_time=0.0)
def arrival():
   global arrival_count, env, light, queue
   while True:
      arrival_count+= 1
      if light == 'red' or len(queue):
         queue.append((arrival_count, env.now))
         min=env.now/60
         print("Car #%d arrived and joined the queue at position %d at time ""%.1f""sec or ""%.1f""mins"%(arrival_count, len(queue), env.now,min))
      else:
         min=env.now/60
         print("Car #%d arrived to a green light with no cars waiting at time ""%.1f""sec or ""%.1f""mins"%(arrival_count, env.now,min))
         W_stats.count+= 1
         #next arival
      yield env.timeout( random.exponential(t_interarrival_mean))
def departure():
     global env, queue
     while True:
      car_number, t_arrival= queue.popleft()
      min=env.now/60
      print("Car #%d departed at time %.1f""sec or"" %.1f""mins"", leaving %d cars in the queue."%(car_number, env.now,min, len(queue)))
      W_stats.count+= 1
      W_stats.waiting_time+= env.now - t_arrival
      if light == 'red' or len(queue) == 0:
         return
      delay= random.triangular(left=t_depart_left, mode=t_depart_mode,right=t_depart_right)
      yield env.timeout(delay)
def light():
   global env, light
   while True:
      light= 'green'
      min=env.now/60
      print("\nThe light turned green at time %.1f.""sec or ""%.1f""mins"%(env.now,min))
      if len(queue):
         delay= random.triangular(left=t_depart_left, mode=t_depart_mode,right=t_depart_right)
         start_delayed(env, departure(), delay=delay)
      yield env.timeout(t_green)
      light= 'red'
      min=env.now/60
      print("\nThe light turned red at time %.1f.""sec or ""%.1f""mins"% (env.now,min))
      yield env.timeout(t_red)
def monitor():
   global env, Q_stats
   while True:
      Q_stats.count+= 1
      Q_stats.cars_waiting+= len(queue)
      yield env.timeout(1.0)
env= simpy.Environment()
env.process(light())
t_first_arrival= random.exponential(t_interarrival_mean)
start_delayed(env, arrival(), delay=t_first_arrival)
env.process(monitor())
env.run(until=end_time)
print("Mean number of cars waiting: %f"% (Q_stats.cars_waiting / float(Q_stats.count)))
print("Mean waiting time (seconds): %f""secs or ""%f mins"  %((W_stats.waiting_time / float(W_stats.count),(W_stats.waiting_time / float(W_stats.count)/60))))


The light turned green at time 0.0.sec or 0.0mins

The light turned red at time 30.0.sec or 0.5mins
Car #1 arrived and joined the queue at position 1 at time 37.7sec or 0.6mins

The light turned green at time 120.0.sec or 2.0mins
Car #1 departed at time 122.0sec or 2.0mins, leaving 0 cars in the queue.
Car #2 arrived to a green light with no cars waiting at time 124.8sec or 2.1mins

The light turned red at time 150.0.sec or 2.5mins
Mean number of cars waiting: 0.420000
Mean waiting time (seconds): 42.170152secs or 0.702836 mins


In [0]:
import itertools
import random
import simpy
RANDOM_SEED = 100
GAS_STATION_SIZE = 300
THRESHOLD = 5      
FUEL_TANK_SIZE = 75   
FUEL_TANK_LEVEL = [5, 50]
REFUELING_SPEED = 3   
TANK_TRUCK_TIME = 300 
T_INTER = [100, 1000]
SIM_TIME = 10000
def car(name, env, gas_station, fuel_pump):
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s arriving at gas station at %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        yield req
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)
        yield env.timeout(liters_required / REFUELING_SPEED)
        print('%s finished refueling in %.1f seconds.' % (name,env.now - start))
def gas_station_control(env, fuel_pump):
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            print('Calling tank truck at %d' % env.now)
            yield env.process(tank_truck(env, fuel_pump))
        yield env.timeout(15)
def tank_truck(env, fuel_pump):
    yield env.timeout(TANK_TRUCK_TIME)
    print('Tank truck arriving at time %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Tank truck refuelling %.1f liters.' % ammount)
    yield fuel_pump.put(ammount)
def car_generator(env, gas_station, fuel_pump):
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, gas_station, fuel_pump))
print('Gas Station refuelling')
random.seed(RANDOM_SEED)
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))
env.run(until=SIM_TIME)

Gas Station refuelling
Car 0 arriving at gas station at 249.0
Car 0 finished refueling in 13.7 seconds.
Car 1 arriving at gas station at 819.0
Car 1 finished refueling in 19.7 seconds.
Car 2 arriving at gas station at 1708.0
Car 2 finished refueling in 15.0 seconds.
Car 3 arriving at gas station at 2530.0
Car 3 finished refueling in 16.0 seconds.
Car 4 arriving at gas station at 3379.0
Car 4 finished refueling in 12.7 seconds.
Car 5 arriving at gas station at 3922.0
Calling tank truck at 3930
Car 5 finished refueling in 21.0 seconds.
Tank truck arriving at time 4230
Tank truck refuelling 294.0 liters.
Car 6 arriving at gas station at 4841.0
Car 6 finished refueling in 21.0 seconds.
Car 7 arriving at gas station at 5486.0
Car 7 finished refueling in 13.7 seconds.
Car 8 arriving at gas station at 5668.0
Car 8 finished refueling in 22.3 seconds.
Car 9 arriving at gas station at 6037.0
Car 9 finished refueling in 9.7 seconds.
Car 10 arriving at gas station at 6811.0
Car 10 finished refueli

In [0]:
'''Q.1 
Design a simulation regarding registration desk in survey where one desk can handle only 2
clients, use source resource criteria in this problem.'''
import random
import simpy
RANDOM_SEED = 200
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 3.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 30  # Max. customer patience
def source(env, number, interval, counter):
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_survey=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
def customer(env, name, counter, time_in_survey):
    arrive = env.now
    print('%.2f %s: Customer ready to take survey' % (arrive, name))
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.timeout(patience)
        wait = env.now - arrive
        if req in results:
            print('%.2f %s: Waited %.2f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_survey)
            yield env.timeout(tib)
            print('%.2f %s: Finished' % (env.now, name))
        else:
            print('%.2f %s: Surveyed after %.2f' % (env.now, name, wait))
print('Regestration desk')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = simpy.Resource(env, capacity=2)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

ModuleNotFoundError: ignored

In [0]:
'''Q2. Design a simulation regarding traffic light where process interruption is used for some special
case.'''
class TrafficLight(object):
  def __init__(self,env):
    self.env=env
    self.action=env.process(self.run(env))
  def run(self,env):
    while True:
      print("Red light in ",  env.now)
      red_light_duration=90
      try:
        yield self.env.timeout(red_light_duration)
      except(simpy.Interrupt):
        print("Emergency vehicle arriving")
      print("Yellow light in ",  env.now)
      yellow_light_duration=10
      try:
        yield self.env.timeout(yellow_light_duration)
      except(simpy.Interrupt):
        print("Emergency vehicle arriving")
      print("Green light in ",  env.now)
      green_light_duration=20
      try:
        yield self.env.timeout(green_light_duration)
      except(simpy.Interrupt):
         print("Emergency vehicle arriving")
def emergency(env,tl,t):
  yield env.timeout(t)
  tl.action.interrupt()
env=simpy.Environment()
tl=TrafficLight(env)
env.process(emergency(env,tl,180))
env.run(400)

Red light in  0
Yellow light in  90
Green light in  100
Red light in  120
Emergency vehicle arriving
Yellow light in  180
Green light in  190
Red light in  210
Yellow light in  300
Green light in  310
Red light in  330
